In [1]:
import elo
import operator
import pandas

K = 32
elos = {}
record = {}

ledger = [line.strip() for line in open("chess.txt").read().split(",")]
for line in ledger:
    p1, outcome, p2 = line.split(" ")
    if not p1 in elos:
        elos[p1] = [1500,0,0,0]
    if not p2 in elos:
        elos[p2] = [1500,0,0,0]
    pre_1 = elos[p1][0]
    pre_2 = elos[p2][0]
    expected_1 = 1./(10.**(-(pre_1-pre_2)/400.)+1)
    expected_2 = 1./(10.**(-(pre_2-pre_1)/400.)+1)
    new_1 = 0
    new_2 = 0
        
    if outcome == "beat":
        new_1 = pre_1 + K*(1 - expected_1)
        new_2 = pre_2 + K*(0 - expected_2)
        elos[p1][1] += 1
        elos[p2][2] += 1
        
    if outcome == "tied":
        new_1 = pre_1 + K*(.5 - expected_1)
        new_2 = pre_2 + K*(.5 - expected_2)
        elos[p1][3] += 1
        elos[p2][3] += 1
        
    elos[p1][0] = new_1
    elos[p2][0] = new_2
        
sorted(elos.items(), key=operator.itemgetter(1), reverse=True), record

pandas.options.display.precision = 0
df = pandas.DataFrame.from_dict(elos, 'index').sort_values(by=0, ascending=False)
df.columns = ['Elo','W','L','T']
df

,Elo,W,L,T
Nick,1607,8,0,0
Jeremy,1558,10,6,0
Marty,1531,4,2,1
Elise,1518,1,0,0
Grace,1513,2,1,0
Sam,1495,7,8,1
Jason,1487,1,2,1
Paige,1486,0,1,0
Nore,1485,0,1,0
Kevin,1471,1,3,1
